# 環境準備與套件安裝

In [ ]:
!pip install langchain langchain-community pypdf python-docx beautifulsoup4 requests

## 導入核心模組

In [ ]:
# 導入核心模組
from langchain.document_loaders import (
    PyPDFLoader, 
    TextLoader, 
    CSVLoader,
    UnstructuredWordDocumentLoader,
    WebBaseLoader
)
from langchain.schema import Document
import os
import requests
from pathlib import Path

print("所有必要的套件已成功安裝和導入！")

## 建立文件

In [ ]:
# 創建一個示例 PDF 內容（在實際應用中，您會載入真實的 PDF 檔案）
sample_pdf_content = """
AI 產品開發指南

第一章：人工智慧基礎
人工智慧（Artificial Intelligence, AI）是指機器模擬人類智慧行為的能力。
現代 AI 系統主要基於機器學習技術，特別是深度學習。

第二章：產品設計原則
在設計 AI 產品時，需要考慮以下關鍵原則：
1. 用戶體驗優先
2. 資料隱私保護
3. 演算法的可解釋性
4. 系統的可靠性和安全性

第三章：開發流程
AI 產品的開發通常包括以下階段：
- 需求分析與可行性評估
- 資料收集與預處理
- 模型訓練與驗證
- 系統整合與測試
- 部署與監控
"""

# 將內容儲存為文字檔案（模擬 PDF 內容）
with open("ai_product_guide.txt", "w", encoding="utf-8") as f:
    f.write(sample_pdf_content)


### 讀取文件

In [ ]:
# 使用 TextLoader 載入檔案（在實際應用中使用 PyPDFLoader）
loader = TextLoader("ai_product_guide.txt", encoding="utf-8")
documents = loader.load()

print(f"成功載入文件：{len(documents)} 個")
print(f"文件類型：{type(documents[0])}")
print(f"文件內容長度：{len(documents[0].page_content)} 字符")
print(f"文件元資料：{documents[0].metadata}")
print("\n--- 文件內容預覽 ---")
print(documents[0].page_content[:200] + "...")

## 處理網頁內容

### 創建一個模擬的網頁內容載入器

In [ ]:
# 創建一個模擬的網頁內容載入器
class MockWebLoader:
    def __init__(self, url):
        self.url = url
    
    def load(self):
        # 模擬不同網站的內容
        mock_content = {
            "company_policy": """
            <html>
            <head><title>公司政策手冊</title></head>
            <body>
                <h1>員工行為準則</h1>
                <h2>工作時間政策</h2>
                <p>標準工作時間為每日8小時，週一至週五上午9點至下午6點。</p>
                
                <h2>遠端工作規定</h2>
                <p>員工可申請彈性工作安排，包括部分時間的遠端工作。</p>
                <ul>
                    <li>需提前一週申請</li>
                    <li>每週最多3天遠端工作</li>
                    <li>必須保持與團隊的有效溝通</li>
                </ul>
                
                <h2>休假政策</h2>
                <p>全職員工每年享有20天帶薪年假。</p>
            </body>
            </html>
            """,
            "tech_docs": """
            <html>
            <head><title>API 技術文件</title></head>
            <body>
                <h1>RESTful API 使用指南</h1>
                <h2>認證方式</h2>
                <p>所有 API 請求都需要在標頭中包含有效的 API 金鑰。</p>
                <code>Authorization: Bearer YOUR_API_KEY</code>
                
                <h2>常用端點</h2>
                <h3>用戶管理</h3>
                <p>GET /api/users - 獲取用戶列表</p>
                <p>POST /api/users - 創建新用戶</p>
                
                <h3>資料查詢</h3>
                <p>GET /api/data - 查詢資料</p>
                <p>支援的參數：limit, offset, filter</p>
            </body>
            </html>
            """
        }
        
        # 根據 URL 返回對應的模擬內容
        if "policy" in self.url:
            content = mock_content["company_policy"]
        else:
            content = mock_content["tech_docs"]
        
        # 簡單的 HTML 標籤清理
        import re
        text_content = re.sub(r'<[^>]+>', ' ', content)
        text_content = re.sub(r'\s+', ' ', text_content).strip()
        
        return [Document(
            page_content=text_content,
            metadata={"source": self.url, "type": "webpage"}
        )]

### 載入不同類型的網頁內容

In [ ]:
policy_loader = MockWebLoader("https://company.com/policy")
tech_loader = MockWebLoader("https://company.com/tech-docs")

policy_docs = policy_loader.load()
tech_docs = tech_loader.load()

In [ ]:
print("=== 政策文件 ===")
print(f"來源：{policy_docs[0].metadata['source']}")
print(f"內容長度：{len(policy_docs[0].page_content)} 字符")
print(f"內容預覽：{policy_docs[0].page_content[:150]}...")

print("\n=== 技術文件 ===")
print(f"來源：{tech_docs[0].metadata['source']}")
print(f"內容長度：{len(tech_docs[0].page_content)} 字符")
print(f"內容預覽：{tech_docs[0].page_content[:150]}...")

## 處理結構化資料

In [ ]:
# 創建示例 CSV 資料
csv_content = """產品名稱,類別,價格,描述,上市日期
智慧手錶 Pro,穿戴裝置,299,具備健康監測和運動追蹤功能的智慧手錶,2024-01-15
無線耳機 X1,音響設備,199,主動降噪無線藍牙耳機，電池續航24小時,2024-02-20
智慧音箱 Home,智慧家居,149,支援語音控制的智慧音箱，內建AI助理,2024-03-10
平板電腦 Tab,電腦設備,599,10吋高解析度觸控螢幕，適合工作和娛樂,2024-01-30
健身追蹤器,穿戴裝置,99,輕量化設計，專注於運動數據追蹤,2024-04-05"""

# 儲存為 CSV 檔案
with open("products.csv", "w", encoding="utf-8") as f:
    f.write(csv_content)

# 使用 CSVLoader 載入資料
csv_loader = CSVLoader("products.csv")
csv_documents = csv_loader.load()

print(f"從 CSV 載入了 {len(csv_documents)} 個產品記錄")
print("\n=== 產品記錄範例 ===")
for i, doc in enumerate(csv_documents[:3]):  # 顯示前3個記錄
    print(f"\n產品 {i+1}:")
    print(f"內容：{doc.page_content}")
    print(f"來源：{doc.metadata}")

## 整合多來源資料

In [ ]:
# 整合所有載入的文件
all_documents = []

# 添加不同來源的文件
all_documents.extend(documents)      # 技術指南
all_documents.extend(policy_docs)    # 公司政策  
all_documents.extend(tech_docs)      # API 文件
all_documents.extend(csv_documents)  # 產品資料

# 為每個文件添加類型標識
for doc in documents:
    doc.metadata["document_type"] = "technical_guide"

for doc in policy_docs:
    doc.metadata["document_type"] = "company_policy"
    
for doc in tech_docs:
    doc.metadata["document_type"] = "api_documentation"
    
for doc in csv_documents:
    doc.metadata["document_type"] = "product_catalog"

In [ ]:
# 顯示整合結果
print(f"知識庫總計包含 {len(all_documents)} 個文件")
print("\n=== 文件類型統計 ===")
doc_types = {}
for doc in all_documents:
    doc_type = doc.metadata.get("document_type", "unknown")
    doc_types[doc_type] = doc_types.get(doc_type, 0) + 1

for doc_type, count in doc_types.items():
    print(f"{doc_type}: {count} 個文件")



In [ ]:
# 清理暫存檔案
import os
for file in ["ai_product_guide.txt", "products.csv"]:
    if os.path.exists(file):
        os.remove(file)
        
print("\n文件載入和整合完成！")

# 分割策略

## 準備檔案進行示範

In [ ]:
# 準備一份較長的示例文件來演示分割效果
long_document_content = """
# RAG 系統架構設計完整指南

## 第一章：系統概述
RAG（Retrieval-Augmented Generation）系統是一種結合檢索和生成技術的先進 AI 架構。它通過在生成回答之前先檢索相關資訊，有效解決了大型語言模型的知識局限性問題。

### 1.1 核心組件
RAG 系統主要包含以下核心組件：

#### 1.1.1 文件載入器
負責從各種來源載入和預處理文件，支援 PDF、Word、網頁等多種格式。文件載入器需要處理編碼問題、格式轉換和內容清理。
#### 1.1.2 文字分割器
將長文件分割成適當大小的片段。分割策略需要考慮語義完整性、片段大小和重疊程度。常用的分割方法包括固定長度分割、段落分割和語義分割。
#### 1.1.3 嵌入模型
將文字片段轉換為高維向量表示。嵌入模型的選擇會影響檢索的準確性。常用的模型包括 OpenAI 的 text-embedding-ada-002、Sentence-BERT 等。
#### 1.1.4 向量資料庫
儲存和檢索文件片段的向量表示。需要支援快速的相似度搜尋和大規模資料儲存。主流選擇包括 Chroma、Pinecone、Weaviate 等。
#### 1.1.5 檢索器
根據使用者查詢檢索最相關的文件片段。檢索策略可以是簡單的相似度搜尋，也可以是複雜的混合檢索方法。
#### 1.1.6 生成模型
基於檢索到的上下文生成最終回答。需要能夠理解和整合多個資訊來源，生成準確、相關的回答。

## 第二章：實作指南
### 2.1 環境準備
在開始實作 RAG 系統之前，需要準備開發環境。建議使用 Python 3.8 以上版本，安裝必要的套件包括 LangChain、OpenAI、Chroma 等。

### 2.2 資料準備
資料準備是 RAG 系統成功的關鍵。需要收集高品質的文件，進行適當的預處理，包括去除雜訊、標準化格式、處理特殊字符等。

### 2.3 系統架構
設計良好的系統架構能夠確保 RAG 系統的可擴展性和維護性。建議採用模組化設計，將不同功能分離到獨立的組件中。

## 第三章：優化策略
### 3.1 檢索優化
檢索是 RAG 系統的核心環節，直接影響最終回答的品質。可以通過調整相似度閾值、使用混合檢索、實作重新排序等方法來優化檢索效果。

### 3.2 生成優化
生成優化主要涉及提示工程和模型調整。好的提示模板能夠引導模型生成更準確、更有用的回答。

### 3.3 系統監控
持續監控系統性能，包括回答品質、回應時間、資源使用等指標，是保證 RAG 系統穩定運行的重要手段。
"""

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import matplotlib.pyplot as plt

# 創建文件物件
long_document = Document(
    page_content=long_document_content,
    metadata={"source": "rag_guide.md", "type": "technical_documentation"}
)

print(f"原始文件長度：{len(long_document.page_content)} 字符")
print(f"原始文件行數：{len(long_document.page_content.splitlines())} 行")


### 分割實作

In [ ]:
# 1. 固定長度分割（最基礎的方法）
basic_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,           # 每個片段500字符
    chunk_overlap=50,         # 片段間重疊50字符
    length_function=len,      # 使用字符數計算長度
    separators=["\n\n", "\n", " ", ""]  # 分割優先順序
)

basic_chunks = basic_splitter.split_documents([long_document])

print("=== 基礎分割結果 ===")
print(f"分割成 {len(basic_chunks)} 個片段")
for i, chunk in enumerate(basic_chunks[:3]):  # 顯示前3個片段
    print(f"\n--- 片段 {i+1} ---")
    print(f"長度：{len(chunk.page_content)} 字符")
    print(f"內容預覽：{chunk.page_content[:100]}...")
    print(f"元資料：{chunk.metadata}")


### 語義分割實作

In [ ]:
# 2. 語義感知分割
semantic_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,           # 較大的片段允許更完整的語義單位
    chunk_overlap=100,        # 更大的重疊確保語義連續性
    length_function=len,
    separators=["## ", "### ", "\n\n", "\n", " ", ""]  # 優先在標題處分割
)

semantic_chunks = semantic_splitter.split_documents([long_document])

print("=== 語義感知分割結果 ===")
print(f"分割成 {len(semantic_chunks)} 個片段")

for i, chunk in enumerate(semantic_chunks):
    print(f"\n--- 片段 {i+1} ---")
    print(f"長度：{len(chunk.page_content)} 字符")
    
    # 檢查片段是否包含完整的語義單位
    content_preview = chunk.page_content.strip()
    if content_preview.startswith('#'):
        print(f"類型：標題開始的片段")
    elif content_preview.startswith('###'):
        print(f"類型：子標題片段")
    else:
        print(f"類型：內容片段")
    
    print(f"開始：{content_preview[:80]}...")
    if i < 3:  # 只顯示前3個片段的完整內容
        print(f"完整內容：\n{content_preview[:300]}...")


## 分割品質分析

In [ ]:
# 分割品質分析函數
def analyze_chunks(chunks, title="分割分析"):
    print(f"\n=== {title} ===")
    
    # 基本統計
    lengths = [len(chunk.page_content) for chunk in chunks]
    print(f"總片段數：{len(chunks)}")
    print(f"平均長度：{sum(lengths)/len(lengths):.1f} 字符")
    print(f"最短片段：{min(lengths)} 字符")
    print(f"最長片段：{max(lengths)} 字符")
    
    # 長度分佈分析
    length_ranges = {
        "很短 (<200)": len([l for l in lengths if l < 200]),
        "短 (200-400)": len([l for l in lengths if 200 <= l < 400]),
        "中等 (400-600)": len([l for l in lengths if 400 <= l < 600]),
        "長 (600-800)": len([l for l in lengths if 600 <= l < 800]),
        "很長 (>800)": len([l for l in lengths if l >= 800])
    }
    
    print("\n長度分佈：")
    for range_name, count in length_ranges.items():
        percentage = (count / len(chunks)) * 100
        print(f"  {range_name}: {count} 個 ({percentage:.1f}%)")
    
    # 語義完整性檢查
    complete_units = 0
    for chunk in chunks:
        content = chunk.page_content.strip()
        # 檢查是否以標題開始或包含完整段落
        if (content.startswith('#') or 
            content.count('\n\n') >= 1 or 
            len(content.split('。')) >= 2):
            complete_units += 1
    
    print(f"\n語義完整性：{complete_units}/{len(chunks)} ({(complete_units/len(chunks)*100):.1f}%) 片段看起來語義完整")
    
    return {
        "total_chunks": len(chunks),
        "avg_length": sum(lengths)/len(lengths),
        "length_distribution": length_ranges,
        "semantic_completeness": complete_units/len(chunks)
    }

# 比較不同分割策略的效果
basic_analysis = analyze_chunks(basic_chunks, "基礎分割分析")
semantic_analysis = analyze_chunks(semantic_chunks, "語義感知分割分析")


### 特定類型優化分割

In [ ]:
# 針對程式碼文件的分割器
def create_code_splitter():
    return RecursiveCharacterTextSplitter(
        chunk_size=600,
        chunk_overlap=50,
        separators=["\n\nclass ", "\n\ndef ", "\n\n```", "\n\n", "\n", " "]
    )

# 針對FAQ文件的分割器  
def create_faq_splitter():
    return RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=30,
        separators=["\nQ:", "\n問：", "\n答：", "\nA:", "\n\n", "\n"]
    )

# 針對API文件的分割器
def create_api_splitter():
    return RecursiveCharacterTextSplitter(
        chunk_size=700,
        chunk_overlap=80,
        separators=["\n## Endpoint", "\n### ", "\n\n", "\n", " "]
    )

# 示範針對技術文件的最佳分割策略
technical_splitter = RecursiveCharacterTextSplitter(
    chunk_size=750,           # 技術文件通常需要更多上下文
    chunk_overlap=75,         # 較大重疊保持技術概念的連續性
    separators=[
        "\n## ",              # 主要章節
        "\n### ",             # 子章節
        "\n#### ",            # 小節
        "\n\n",               # 段落
        "\n",                 # 行
        " "                   # 詞
    ]
)

technical_chunks = technical_splitter.split_documents([long_document])
technical_analysis = analyze_chunks(technical_chunks, "技術文件優化分割")

# 顯示最終推薦的分割結果
print("\n=== 推薦分割策略的片段範例 ===")
for i, chunk in enumerate(technical_chunks[:2]):
    print(f"\n片段 {i+1}：")
    print(f"長度：{len(chunk.page_content)} 字符")
    print(f"內容：\n{chunk.page_content[:400]}...")
    print("-" * 60)


### 動態分割

In [ ]:
# 智慧參數調整函數
def get_optimal_chunk_size(document_length, document_type):
    """根據文件長度和類型推薦最佳分割參數"""
    
    base_configs = {
        "technical_documentation": {"base_size": 750, "overlap_ratio": 0.1},
        "company_policy": {"base_size": 500, "overlap_ratio": 0.15},
        "api_documentation": {"base_size": 600, "overlap_ratio": 0.12},
        "product_catalog": {"base_size": 300, "overlap_ratio": 0.2}
    }
    
    config = base_configs.get(document_type, {"base_size": 600, "overlap_ratio": 0.12})
    
    # 根據文件長度調整
    if document_length < 2000:
        size_multiplier = 0.7
    elif document_length > 10000:
        size_multiplier = 1.3
    else:
        size_multiplier = 1.0
    
    chunk_size = int(config["base_size"] * size_multiplier)
    chunk_overlap = int(chunk_size * config["overlap_ratio"])
    
    return chunk_size, chunk_overlap

# 為我們之前載入的不同類型文件應用最佳分割
optimized_chunks = []

for doc in all_documents:
    doc_type = doc.metadata.get("document_type", "technical_documentation")
    doc_length = len(doc.page_content)
    
    chunk_size, chunk_overlap = get_optimal_chunk_size(doc_length, doc_type)
    
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n## ", "\n### ", "\n\n", "\n", " ", ""]
    )
    
    chunks = splitter.split_documents([doc])
    
    # 為每個片段添加分割資訊
    for chunk in chunks:
        chunk.metadata["chunk_size_used"] = chunk_size
        chunk.metadata["chunk_overlap_used"] = chunk_overlap
        chunk.metadata["original_doc_type"] = doc_type
    
    optimized_chunks.extend(chunks)

print(f"使用動態參數分割後，總共產生 {len(optimized_chunks)} 個片段")

# 按文件類型統計分割結果
type_stats = {}
for chunk in optimized_chunks:
    doc_type = chunk.metadata["original_doc_type"]
    if doc_type not in type_stats:
        type_stats[doc_type] = {"count": 0, "total_length": 0}
    type_stats[doc_type]["count"] += 1
    type_stats[doc_type]["total_length"] += len(chunk.page_content)

print("\n=== 各類型文件分割統計 ===")
for doc_type, stats in type_stats.items():
    avg_length = stats["total_length"] / stats["count"]
    print(f"{doc_type}:")
    print(f"  片段數量：{stats['count']}")
    print(f"  平均長度：{avg_length:.1f} 字符")
